# Principal Component Analysis (PCA)


## Table of contents
* [0. Libraries and helper functions](#libraries)
* [1. TwoD dataset rotated into ThreeD plus noise](#twod)
* [2. Eigendigits](#eigendigits)
* [3. Kernel PCA](#kpca)

Note that in this notebook the data is given as a matrix of row vectors, i.e.
$$ X = [x_1, \ldots, x_n]^T \in \mathbb{R}^{n\times d}$$

<a class="anchor" id="libraries"></a>
## 0. Libraries and helper functions

In [ ]:
from ml_solutions            import pca, pca_project, kpca
#def pca(x, d):
#     inputs:
#       `x` is (n,D)-matrix or (N,h,w) or ...
#       `d` is number of PCs
#     outputs:
#       `y` is (n,d)-matrix
#       `Lambda` is (d)-vector
#       `V` is (d,D)-matrix or (d,h,w) or ...
#     ...
#     return y, Lambda, V

# def pca_project(x, V):
#     inputs:
#       `x` is (n,D)-matrix or (N,h,w) or ...
#       `V` is (d,D)-matrix or (d,h,w) or ...
#     outputs:
#       `z` is (n,D)-matrix or (N,h,w) or ...
#     ...
#     return z

# def kpca(x, d, k):
#     inputs:
#       `x` is (n,D)-matrix or (N,h,w) or ...
#       `d` is number of PCs
#       `k` is kernel function 
#     outputs:
#       `y` is (n,d)-matrix
#       `Lambda` is (d)-vector
#       `V` is (d,n)-matrix
#     ...
#     return y, Lambda, V

In [ ]:
import numpy as np
from numpy.random import seed, randn, rand
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import torchvision.datasets

In [ ]:
#iterable = lambda x: hasattr(x, '__iter__')
#def rand(*args):
#    if len(args) == 1 and iterable(args[0]): # allows rand([3,4])
#        return np.random.rand(*args[0])      # allows rand(3,4)
#    return np.random.rand(*args)
#def randn(*args):
#    if len(args) == 1 and iterable(args[0]):
#        return np.random.randn(*args[0])  # allows randn([3,4])
#    return np.random.randn(*args)         # allows randn(3,4)
#def ones(*args):
#    if len(args) == 1:
#        return np.ones(args[0])   # allows ones([3,4])
#    return np.ones(args)          # allows ones(3,4)
#def zeros(*args):
#    if len(args) == 1:
#        return np.zeros(args[0])  # allows zeros([3,4])
#    return np.zeros(args)         # allows zeros(3,4)

In [ ]:
def random_rotation(d, D):
    # random rotation matrix 
    # input d dimensions
    # output D dimensions
    # e.g. 
    #   Z = randn(n, d)
    #   U = random_rotation(d, D)
    #   X = Z @ U
    # to rotate `d` dimensions randomly into `D`
    if d is None: d = D
    if D<d: raise "D must not be smaller than d"
    # sample a random rotation matrix
    X = np.random.randn(d, D)
    # return economy sized matrices
    return np.linalg.svd(X, full_matrices=False)[2]
U = random_rotation(2,3)
print(U)
print(U.T @ U)    # is identity if D==d
print(U @ U.T)    # must be identity

In [ ]:
help(np.linalg.eig)    # for square matrices
help(np.linalg.eigh)   # for symmetric square matrices

In [ ]:
A = randn(3,3)
A = A + A.T
Lambda, V = np.linalg.eigh(A)
print("Lambda = \n", Lambda)
print("V = \n", V)
print("A = \n", A)
print("V @ Diag(Lambda) @ V.T = \n", V @ np.diag(Lambda) @ V.T)

In [ ]:
V.T @ V

In [ ]:
help(np.linalg.svd)

In [ ]:
A = randn(3,4)
#help(np.linalg.svd)
[U,S,V] = np.linalg.svd(A, full_matrices=False)
print(U.shape, S.shape, V.shape)
#print((U * S) @ V)
print(U @ np.diag(S) @ V)
print(A)

In [ ]:
print(V @ V.T)  # Harmeling
print(V.T @ V)  #

In [ ]:
help(np.linalg.svd)

In [ ]:
def show_3d(X):
    fig = px.scatter_3d(x=X[0], y=X[1], z=X[2])
    return fig

In [ ]:
# based on https://stackoverflow.com/questions/66789390/draw-an-arrow-between-two-specific-points-in-a-scatter-plot-with-plotly-graph-ob
def add_arrow_3d(fig, FROM, TO, color='rgb(255,0,0)'):
    # draw an arrow from x0 to x1
    # draw a line
    fig.add_trace(go.Scatter3d(x=[FROM[0],TO[0]],
                               y=[FROM[1],TO[1]],
                               z=[FROM[2],TO[2]],
                               mode='lines',
                               line = dict(width=5, color=color)))
    # draw the arrow head
    arrow_tip_ratio = 1.0
    arrow_starting_ratio = 0.98
    v = [t-f for (t,f) in zip(TO, FROM)]
    norm_v = np.sqrt(v[0]**2 + v[1]**2 + v[2]**2)
    fig.add_trace(go.Cone(
        x=[FROM[0] + arrow_starting_ratio*v[0]],
        y=[FROM[1] + arrow_starting_ratio*v[1]],
        z=[FROM[2] + arrow_starting_ratio*v[2]],
        u=[arrow_tip_ratio*v[0]/norm_v],
        v=[arrow_tip_ratio*v[1]/norm_v],
        w=[arrow_tip_ratio*v[2]/norm_v],
        showlegend=False,
        showscale=False,
        colorscale=[[0, color], [1, color]]
        ))


<a class="anchor" id="twod"></a>
## 1. TwoD dataset rotated into ThreeD plus noise

In [ ]:
n, d, D = 100, 2, 3
seed(0)
Z = randn(n, d)
Z[:,1] *= 2.0      # rescale one of the axis
U = random_rotation(d, D)
sigma_sq = 0.03        # noise variance
X = Z @ U + sigma_sq * randn(n, D)

# show the data
fig = px.scatter_3d(x=X[:,0], y=X[:,1], z=X[:,2], color=X[:,0])
fig.show()

In [ ]:
# run PCA on it to the PCA direction
y, Lambda, V = pca(X, 3)
mu = X.mean(axis=0)
# show the results
fig = px.scatter_3d(x=X[:,0], y=X[:,1], z=X[:,2])
fig.update_traces(marker={'size': 5})
add_arrow_3d(fig, mu, mu + 2*np.sqrt(Lambda[0])*V[0])
add_arrow_3d(fig, mu, mu + 2*np.sqrt(Lambda[1])*V[1])
add_arrow_3d(fig, mu, mu + 2*np.sqrt(Lambda[2])*V[2])
fig.update_layout(width=800, height=800)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
#fig.write_image('two_d_in_three_d.pdf')
Lambda

<a class="anchor" id="eigendigits"></a>
## 2. Eigendigits

In [ ]:
digits = torchvision.datasets.MNIST(root='', download=True)
x = digits.data.numpy()
y = digits.targets.numpy()

In [ ]:
zeros = x[y==0]
ones  = x[y==1]
twos  = x[y==2]
twos.shape

In [ ]:
def digits_gallery(x, columns=10):
    z = x
    n = z.shape[0]    # number of digits
    m = n % columns   # how many extra in the last row
    h,w = x.shape[1], x.shape[2]
    if m > 0:
        z = np.vstack([z, np.zeros((columns-m,h,w))])  # fill with zeros
    n = z.shape[0]
    rows = n//columns
    z = z.reshape(rows, columns, h, w)
    z = z.transpose([0,2,1,3])
    z = z.reshape(rows*h, columns*w)
    return z.astype(float)
def plot_digits(x, columns=10, width=800, height=800):
    fig = px.imshow(digits_gallery(x, columns), 
                 color_continuous_scale='Greys')
    fig.update_layout(width=width, height=height)
    return fig
fig = plot_digits(zeros[:100],10)
#fig.write_image('mnist_digits.pdf')
fig.show()

In [ ]:
# run pca on the digits
d = 100
y, Lambda, V = pca(zeros, d)
fig = plot_digits(V, 10)
#fig.write_image('mnist_eigendigits.pdf')
fig.show()

In [ ]:
px.line(Lambda, title='Eigenvalue spectrum')

In [ ]:
# reconstructing from a few components

# use einsum for this!!!
d_reconstruction = 3   # how many eigenvectors to use
plot_digits(pca_project(twos[:100], V[:d_reconstruction]), 10)
# fig = make_subplots(rows=1, cols=2)
#     fig.add_trace(

<a class="anchor" id="kpca"></a>
## 3. Kernel PCA

In [ ]:
# here we assume that each data point is a row!!!
def polynomial_kernel(x, x_prime, p, b=0):
    return (x @ x_prime.T + b)**p
def linear_kernel(x, x_prime):
    return polynomial_kernel(x, x_prime, 1, 0)
def gaussian_kernel(x, x_prime, sigma_sq=1.0):
    m,n = x.shape[0], x_prime.shape[0]
    all_distances_sq = (x**2).sum(1).reshape(m,1) \
        + (x_prime**2).sum(1).reshape(1,n)        \
        - 2*(x@x_prime.T)
    return np.exp(-all_distances_sq / (2*sigma_sq))

In [ ]:
def plot_pca(x, y, columns=2):
    d = y.shape[1]    # number of principal components
    rows = (d//columns) + (d%columns > 0)
    fig = make_subplots(rows=rows, cols=columns)
    i = 0
    for r in range(rows):
        for c in range(columns):
            fig.add_trace(go.Scatter(x=x[:,0], y=x[:,1], 
                             marker=dict(color=y[:,i],
                                         colorscale="Viridis"), 
                             mode='markers'), 
                  row=r+1, col=c+1)
            fig.update_yaxes(scaleanchor='x', scaleratio=1)
            i += 1
            if i >= d:
                break
    return fig

In [ ]:
# toy data: arc
n = 1000
x = 2*rand(n, 2)-1
x[:,1] = x[:,0]**2
x += 0.03 * randn(n, 2)
k = lambda x, x_prime: gaussian_kernel(x, x_prime, 1.0)
#k = lambda x, x_prime: linear_kernel(x, x_prime)
y, Lambda, V = kpca(x, 4, k)
plot_pca(x, y, columns=2)

In [ ]:
# toy data: cluster
n = 1000
x = randn(n, 2)
x[:n//2,0] += 10.0
x[n//4:3*n//4,1] += 10.0
k = lambda x, x_prime: gaussian_kernel(x, x_prime, 3.0)
#k = lambda x, x_prime: linear_kernel(x, x_prime)
y, Lambda, V = kpca(x, 4, k)
plot_pca(x, y, columns=2)

In [ ]:
# toy data: sheet of paper
n = 1000
x = rand(n, 2)
x[:,0] *= 5.0
k = lambda x, x_prime: gaussian_kernel(x, x_prime, 1.0)
#k = lambda x, x_prime: linear_kernel(x, x_prime)
y, Lambda, V = kpca(x, 25, k)
plot_pca(x, y, columns=5)